# 0.Tensor
在本章中，将包括以下内容
1. Tensor的直观理解
2. Tensor的基本操作：
    * 生成
    * 基本运算
    * 与Numpy的转换
    * GPU计算
3. 利用Tensor完成简单的3层神经网络

## Tensor直观理解
* 直观理解Tensor就类似Numpy中的array，是一个n维数组，专门用于在pytorch中做**数值计算**，不涉及深度学习网络结构、计算图、梯度计算等。
* 不同于Num平移中的array，torch.Tensor能够使用GPU进行加速，一般GPU速度是CPU的50倍
* 总结：Tensor是一个单纯的可以用GPU加速的科学计算工具,应与后面介绍的Variable区分开

## Tensor基本操作
### Tensor生成

In [1]:
import torch
import numpy as np

In [2]:
# 生成未初始化矩阵
x = torch.Tensor(5, 3)

# 生成随机矩阵
x = torch.rand(5, 3)

# 得到Tensor size
print('x.size(): ', x.size())

# 指定生成的Tensor的数据类型
dtype = torch.FloatTensor # 使用CPU
# dtype = torch.cuda.FloatTensor # 使用GPU
x = torch.rand(5, 3).type(dtype)
print('x.type(): ',x.type())

('x.size(): ', torch.Size([5, 3]))
('x.type(): ', 'torch.FloatTensor')


### Tensor基本运算
Tensor的运算有多种语法实现
具体计算详见：http://pytorch.org/docs/torch

In [12]:
# 生成Tensor
x = torch.randn(2, 3)
y = torch.randn(2, 3)
print 'x, y:', x, y
print

# 下面用4种语法进行Tensor加法
# 语法1
print 'x + y: ', x + y
print

# 语法2
print 'torch.add(x, y): ', torch.add(x, y)
print 

# 语法3
result = torch.Tensor(2, 3)
torch.add(x, y, out=result)
print 'torch.add(x, y, result): ', result
print 

# 语法4：in-place计算，类似'+='运算，此时直接赋值给y。in-place操作都使用`_`作为后缀。例如，x.copy_(y)
y.add_(x)
print 'y.add_(x)', y

x, y: 
-0.6022  0.2793  0.3447
-0.2408  0.6655  1.4507
[torch.FloatTensor of size 2x3]
 
-0.0998 -0.2380  1.1340
-0.0897  0.8716 -0.5151
[torch.FloatTensor of size 2x3]


x + y:  
-0.7020  0.0413  1.4786
-0.3304  1.5371  0.9357
[torch.FloatTensor of size 2x3]


torch.add(x, y):  
-0.7020  0.0413  1.4786
-0.3304  1.5371  0.9357
[torch.FloatTensor of size 2x3]


torch.add(x, y, result):  
-0.7020  0.0413  1.4786
-0.3304  1.5371  0.9357
[torch.FloatTensor of size 2x3]


y.add_(x) 
-0.7020  0.0413  1.4786
-0.3304  1.5371  0.9357
[torch.FloatTensor of size 2x3]




### Tensor与Numpy进行转换
* torch Tensor可以转换成Numpy arry，反之亦然
* Tensor与numpy进行转换后，两者的内存位置共享，也就是说Tensor的值改变了，对应Numpy的值也改变，反之亦然

In [13]:
# 生成Tensor与numpy arrya
x_tensor = torch.randn(2, 3)
y_numpy = np.random.randn(2, 3)

# Tensor转换为Numpy
x_numpy = x_tensor.numpy()
print 'type(x_numpy):', type(x_numpy)

# Numpy转换为Tensor
y_tensor = torch.from_numpy(y_numpy)
print 'y_tensor.type(): ', y_tensor.type()

type(x_numpy): <type 'numpy.ndarray'>
y_tensor.type():  torch.DoubleTensor


In [20]:
# 各自对应的Tensor与Numpy共享内存，一变俱变
x_tensor.add_(1)
print 'x_tensor after add 1: \n', x_tensor
print
print 'x_numpy after add 1: \n', x_numpy
print

np.add(y_numpy, 2, out=y_numpy)
print 'y_tensor after add 2: ', y_tensor 
print 
print 'y_numpy after add 2: \n', y_numpy
print 

# 如果没有使用in-place运算，则无法共享内存
y_numpy = y_numpy + 1
print 'y_numpy after add 1 without in-place op: \n', y_numpy
print
print 'y_tensor after add 1 without in-place op: ', y_tensor


x_tensor after add 1: 

 7.0147  6.0188  6.8715
 8.2571  6.9253  5.0008
[torch.FloatTensor of size 2x3]


x_numpy after add 1: 
[[ 7.01474714  6.01878738  6.87148237]
 [ 8.25705528  6.92529106  5.00075436]]

y_tensor after add 2:  
 3.0406  3.8906  3.0224
 1.9635  2.9981  0.5178
[torch.DoubleTensor of size 2x3]


y_numpy after add 2: 
[[ 21.04056901  21.89062435  21.02235041]
 [ 19.9634549   20.99806561  18.51780203]]

y_numpy after add 1 without in-place op: 
[[ 22.04056901  22.89062435  22.02235041]
 [ 20.9634549   21.99806561  19.51780203]]

y_tensor after add 1 without in-place op:  
 3.0406  3.8906  3.0224
 1.9635  2.9981  0.5178
[torch.DoubleTensor of size 2x3]



### 在GPU上使用Tensor
上诉的操作中，Tensor都使用CPU进行计算。下面将会使用GPU进行计算。

In [24]:
x = x.cuda()
y = y.cuda()
print 'x + y on GPU: ', x + y

x + y on GPU:  
-1.3042  0.3206  1.8233
-0.5712  2.2025  2.3864
[torch.cuda.FloatTensor of size 2x3 (GPU 0)]



## 用Tensor建立简单的网络
* 完全使用Tensor建立一个3层神经网络
* 需要自己完成梯度计算和梯度更新

In [25]:
from sklearn.datasets import load_boston
from sklearn import preprocessing

dtype = torch.FloatTensor
# dtype = torch.cuda.FloatTensor

# 载入数据，并预处理
X, y = load_boston(return_X_y=True)
X = preprocessing.scale(X[:100,:])
y = preprocessing.scale(y[:100].reshape(-1, 1))

# 定义超参数
data_size, D_input, D_output, D_hidden = X.shape[0], X.shape[1], 1, 50
lr = 1e-5
epoch = 200000

# 转换为Tensor
# X = torch.Tensor(X).type(dtype)
# y = torch.Tensor(y).type(dtype)
X = torch.from_numpy(X).type(dtype)
y = torch.from_numpy(y).type(dtype)

# 定义训练参数
w1 = torch.randn(D_input, D_hidden).type(dtype)
w2 = torch.randn(D_hidden, D_output).type(dtype)

# 进行训练
for i in range(epoch):
    
    # 前向传播
    h = torch.mm(X, w1)                # 计算隐层
    h_relu = h.clamp(min=0)            # relu
    y_pred = torch.mm(h_relu, w2)      # 输出层
    
    # loss计算，使用L2损失函数
    loss = (y_pred - y).pow(2).sum()
    
    if i % 10000 == 0:
        print('epoch: {} loss: {}'.format(i, loss))
    
    # 反向传播，计算梯度
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = X.t().mm(grad_h)

    # 更新计算的梯度
    w1 -= lr * grad_w1
    w2 -= lr * grad_w2

epoch: 0 loss: 55129.8789333
epoch: 10000 loss: 6.49848133915
epoch: 20000 loss: 2.71281233412
epoch: 30000 loss: 1.45664574848
epoch: 40000 loss: 0.869384531962
epoch: 50000 loss: 0.538773518936
epoch: 60000 loss: 0.353727315913
epoch: 70000 loss: 0.247347185427
epoch: 80000 loss: 0.178062128522
epoch: 90000 loss: 0.132103537549
epoch: 100000 loss: 0.0990644682941
epoch: 110000 loss: 0.0731622382732
epoch: 120000 loss: 0.0548861216202
epoch: 130000 loss: 0.0414879448364
epoch: 140000 loss: 0.031505113572
epoch: 150000 loss: 0.0240738952406
epoch: 160000 loss: 0.0184511786115
epoch: 170000 loss: 0.0142171950947
epoch: 180000 loss: 0.010975251201
epoch: 190000 loss: 0.00849652048041
